In [1]:
stream = [0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1]

In [2]:
class LFSR:
    def __init__(self, key, taps):
        d = max(taps)
        assert len(key) == d, "Error: key of wrong size."
        self._s = key
        self._t = [d - t for t in taps]

    def _sum(self, L):
        s = 0
        for x in L:
            s ^^= x
        return s

    def _clock(self):
        b = self._s[0]
        self._s = self._s[1:] + [self._sum(self._s[p] for p in self._t)]
        return b

    def getbit(self):
        return self._clock()


# Crack second LFSR

In [3]:
import itertools
import copy

In [5]:
key_len = 27
key = stream[:key_len]

for b in range(1, key_len + 1): # 改幾個 bits
    print(b)
    for c in itertools.combinations(range(key_len), b):
        key_candidate = copy.deepcopy(key)
        for flip_b in c:
            key_candidate[flip_b] = 1 - key_candidate[flip_b]
        lfsr = LFSR(key_candidate, [27, 26, 25, 22])
        s = [lfsr.getbit() for _ in range(256)]
        matches = sum(a == b for a, b in zip(stream[:256], s))
        
        if matches > int(0.7 * 256):
            print(matches / 256.0, key_candidate)

1
2
3
4
0.746093750000000 [0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1]


KeyboardInterrupt: 

# Crack third LFSR

In [6]:
key_len = 23
key = stream[:key_len]

for b in range(1, key_len + 1): # 改幾個 bits
    print(b)
    for c in itertools.combinations(range(key_len), b):
        key_candidate = copy.deepcopy(key)
        for flip_b in c:
            key_candidate[flip_b] = 1 - key_candidate[flip_b]
        lfsr = LFSR(key_candidate, [23, 22, 20, 18])
        s = [lfsr.getbit() for _ in range(256)]
        matches = sum(a == b for a, b in zip(stream[:256], s))
        
        if matches > int(0.7 * 256):
            print(matches / 256.0, key_candidate)

1
2
3
4
5
6
0.746093750000000 [0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1]


KeyboardInterrupt: 

# Crack first LFSR

In [7]:
lfsr1 = LFSR([0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1], [27, 26, 25, 22])
lfsr2 = LFSR([0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1], [23, 22, 20, 18])

In [8]:
lfsr1_stream = [lfsr1.getbit() for _ in range(256)]
lfsr2_stream = [lfsr2.getbit() for _ in range(256)]

In [154]:
equation_coeff = []

for i, (s, l1s, l2s) in enumerate(zip(stream[:256], lfsr1_stream, lfsr2_stream)):
    if s == l1s and l1s != l2s:
        print(f"stream[{i}] = 1")
        equation_coeff.append((i, 1))
        
    if s == l2s and l1s != l2s:
        print(f"stream[{i}] = 0")
        equation_coeff.append((i, 0))
        
    if len(equation_coeff) >= 22:
        break

stream[4] = 1
stream[7] = 0
stream[12] = 1
stream[14] = 1
stream[17] = 1
stream[18] = 0
stream[19] = 1
stream[20] = 1
stream[22] = 0
stream[24] = 0
stream[26] = 1
stream[27] = 1
stream[28] = 1
stream[30] = 1
stream[31] = 1
stream[34] = 1
stream[35] = 0
stream[39] = 0
stream[40] = 1
stream[41] = 0
stream[42] = 0
stream[43] = 0


In [155]:
equation_coeff

[(4, 1),
 (7, 0),
 (12, 1),
 (14, 1),
 (17, 1),
 (18, 0),
 (19, 1),
 (20, 1),
 (22, 0),
 (24, 0),
 (26, 1),
 (27, 1),
 (28, 1),
 (30, 1),
 (31, 1),
 (34, 1),
 (35, 0),
 (39, 0),
 (40, 1),
 (41, 0),
 (42, 0),
 (43, 0)]

In [156]:
P.<x> = PolynomialRing(GF(2))
P = x^19 + x^5 + x^2 + x + 1
C = companion_matrix(P, format='bottom')

In [157]:
C

[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]

In [231]:
s = var('s', n=19)
sv = vector(s)

for i, out in equation_coeff:
    fv = (C ^ i)[0].change_ring(QQ)
    print(fv * sv - out, end=',\n')

s4 - 1,
s7,
s12 - 1,
s14 - 1,
s17 - 1,
s18,
s0 + s1 + s2 + s5 - 1,
s1 + s2 + s3 + s6 - 1,
s3 + s4 + s5 + s8,
s10 + s5 + s6 + s7,
s12 + s7 + s8 + s9 - 1,
s10 + s13 + s8 + s9 - 1,
s10 + s11 + s14 + s9 - 1,
s11 + s12 + s13 + s16 - 1,
s12 + s13 + s14 + s17 - 1,
s1 + s15 + s16 + s17 + s2 + s3 + s6 - 1,
s16 + s17 + s18 + s2 + s3 + s4 + s7,
s1 + s11 + s3 + s5,
s12 + s2 + s4 + s6 - 1,
s13 + s3 + s5 + s7,
s14 + s4 + s6 + s8,
s15 + s5 + s7 + s9,


In [183]:
R.<s0, s1, s2, s3, s4, s5, s6, s7, s8, s9, s10, s11, s12, s13, s14, s15, s16, s17, s18> = PolynomialRing(GF(2))
J = R.ideal(s4 - 1,
            s7,
            s12 - 1,
            s14 - 1,
            s17 - 1,
            s18,
            s0 + s1 + s2 + s5 - 1,
            s1 + s2 + s3 + s6 - 1,
            s3 + s4 + s5 + s8,
            s10 + s5 + s6 + s7,
            s12 + s7 + s8 + s9 - 1,
            s10 + s13 + s8 + s9 - 1,
            s10 + s11 + s14 + s9 - 1,
            s11 + s12 + s13 + s16 - 1,
            s12 + s13 + s14 + s17 - 1,
            s1 + s15 + s16 + s17 + s2 + s3 + s6 - 1,
            s16 + s17 + s18 + s2 + s3 + s4 + s7,
            s1 + s11 + s3 + s5,
            s12 + s2 + s4 + s6 - 1,
            s13 + s3 + s5 + s7,
            s14 + s4 + s6 + s8,
            s15 + s5 + s7 + s9,
           )
J.variety()

[{s18: 0, s17: 1, s16: 0, s15: 1, s14: 1, s13: 0, s12: 1, s11: 0, s10: 1, s9: 1, s8: 1, s7: 0, s6: 1, s5: 0, s4: 1, s3: 0, s2: 0, s1: 0, s0: 1}]

In [228]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad
import hashlib

In [218]:
key = [1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0] + \
      [0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1] + \
      [0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1]

In [220]:
key_int = 0
for i, b in enumerate(reversed(key)):
    key_int += 2 ^ i * b

In [225]:
sha1 = hashlib.sha1()
sha1.update(str(key_int).encode('ascii'))
key_digest = sha1.digest()[:16]

In [227]:
cipher = AES.new(key_digest, AES.MODE_CBC, bytes.fromhex('cd2832f408d1d973be28b66b133a0b5f'))
unpad(cipher.decrypt(bytes.fromhex('1e3c272c4d9693580659218739e9adace2c5daf98062cf892cf6a9d0fc465671f8cd70a139b384836637c131217643c1')), 16)

b'FLAG{941ae21eb8823b73973fc67ccbf89ce9fb4cd38c}'